In [402]:
import random
import matplotlib.pyplot as plt
import math
import numpy as np
import matplotlib.patches as patches
#random.seed(5918)
#np.random.seed(5918)
import os
import re

from ast import literal_eval as make_tuple

In [306]:
%run Preprocessing.ipynb

In [307]:
def make_map_with_clusters(n_nodes_per_cluster:int, n_clusters:int,cluster_radius = 200):

    max_x = 1000
    max_y = 1000
    
    turbine_id = 1
    
    grid = {}
    
    grid[0] = (0,0)
    
    cluster_centers = []
    cs = []
    
    for _ in range(n_clusters):
        angle = random.uniform(0,math.pi/2)
        
        apexx = int(cluster_radius*math.cos(angle))
        apexy = int(cluster_radius*math.sin(angle))
        cx = round(random.uniform(apexx, max_x))
        cy = round(random.uniform(apexy,max_y))
        cluster_centers.append((cx,cy))
        grid[turbine_id] = (cx,cy)
        cs.append(turbine_id)
        turbine_id+=1
        
    for cluster in cluster_centers:
        cx,cy = cluster
    
        for _ in range(n_nodes_per_cluster):
            x = int(random.gauss(cx,50))
            y = int(random.gauss(cy,50))
            
            x = max(0,min(x,max_x))
            y = max(0,min(y,max_y))
            
            grid[turbine_id] = (x,y)
            turbine_id +=1
    

    return grid,cs,cluster_centers

    
    
    

In [308]:
def generate_jobs(nodes:dict,n_chargers): #The nodes here should only be the OTWs
    """This function generates jobs for the problem. The crew_needed for each job states how much of the passanger capacity the crew for this job takes in the vessel. The priority property is used
    in the objective function in the case that a job is left unserved. A higher priority yields a larger penalty. This function can be expanded to give more parameters to the jobs-

    Args:
        nodes (dict): All the nodes in the problem
        n_chargers (int): The number of chargers in the problem. This is used to find the correct index of the OTW-nodes

    Returns:
        jobs (dict): All the jobs with turbine_id as key and (crew_needed, priority) as value. 
    """
    #np.random.seed(5918)
    n_nodes = len(nodes)
    
    crew_needed = np.clip(np.random.normal(loc=5, scale=2, size=n_nodes).astype(int), a_min=1, a_max=None)
    crew_needed = crew_needed.tolist()

    
    jobs = {}
    for i in range(n_chargers+1,n_nodes):
        tag = i
        job = crew_needed.pop()
        jobs[tag] = job

    return jobs


In [309]:
'''test = make_a_test_map(10)
tour1, len1 = make_tour(test,3)
tour2, len2 = make_tour(test,2)
cluster_test, centers = make_a_test_map_with_clusters(3,3,35)
clustertour1, clustertourlength1 = make_tour(cluster_test,4)
clustertour2, clustertourlength2 = make_tour(cluster_test,3)
'''

'test = make_a_test_map(10)\ntour1, len1 = make_tour(test,3)\ntour2, len2 = make_tour(test,2)\ncluster_test, centers = make_a_test_map_with_clusters(3,3,35)\nclustertour1, clustertourlength1 = make_tour(cluster_test,4)\nclustertour2, clustertourlength2 = make_tour(cluster_test,3)\n'

In [310]:
'''clusterplot = show_the_grid(cluster_test,len(centers))
plot_tour_on_map(clusterplot,cluster_test,clustertour1,clustertourlength1,"Tour 1")
plot_tour_on_map(clusterplot,cluster_test,clustertour2,clustertourlength2,"Tour 2",color = 'purple')
plot_cluster_centers(clusterplot,centers,30)
print(clustertour2)'''

'clusterplot = show_the_grid(cluster_test,len(centers))\nplot_tour_on_map(clusterplot,cluster_test,clustertour1,clustertourlength1,"Tour 1")\nplot_tour_on_map(clusterplot,cluster_test,clustertour2,clustertourlength2,"Tour 2",color = \'purple\')\nplot_cluster_centers(clusterplot,centers,30)\nprint(clustertour2)'

In [311]:
def montecarlolength():
    '''
    I want this function to make 10 grids, and for each grid to make 10000 tours of tourlength tourlength.
    I should then check that each node is visited evenly, and primarily it should calculate the average length of the tours
    '''
    avgs = np.zeros(4)
    i_s = 10
    j_s = 100000
    for n in range(1,5):
        
        for i in range(i_s):
            
            curr_map = make_a_test_map(15)
            
            for j in range(j_s):
                tour, tourlength = make_tour(curr_map,n)
                avgs[n-1] += tourlength/(i_s*j_s)
    
    
    return avgs
        

In [312]:
def montecarlolength_cluster():
    '''
    I want this function to make 10 grids, and for each grid to make 10000 tours of tourlength tourlength.
    I should then check that each node is visited evenly, and primarily it should calculate the average length of the tours
    '''
    avgs = np.zeros(4)
    i_s = 10
    j_s = 100000
    for n in range(1,5):
        
        for i in range(i_s):
            
            curr_map,_, _ = make_map_with_clusters(6,1,20)
            
            for j in range(j_s):
                tour, tourlength = make_tour(curr_map,n)
                avgs[n-1] += tourlength/(i_s*j_s)
    
    
    return avgs

In [313]:
#lengths = montecarlolength()

In [314]:
#cluster_lengths = montecarlolength_cluster()

In [315]:
def vessel_types():
    vessels = {}
    #ID: The keys in the dictionary is the vessel id.
    battery_range = [2000,3000,4500] # I think these ranges should be okay for now since it allows the medium and big vessel type to go to more than one cluster
    #coeffecient = [] #kWh / km 
    operational_cost = [1, 2, 3.5] #cost per km
    speed = [20,22,25] # Knots. In km/h: 37,41,46
    passenger_capacity = [10,15,25] # This will be modified depending on the avg size of the crews.
    fixed_cost = [5000,10000,17000]
    
    
    vessels[1] = [1,battery_range[0],operational_cost[0],speed[0], passenger_capacity[0],fixed_cost[0]]
    vessels[2] = [2,battery_range[1],operational_cost[1],speed[1], passenger_capacity[1],fixed_cost[1]]
    vessels[3] = [3,battery_range[2],operational_cost[2],speed[2], passenger_capacity[2],fixed_cost[2]]
    
    return vessels
    
    
    

In [316]:
def all_nodes(jobs):
    
    nodes = []
    
    for job in jobs.keys():
        nodes.append((job,0))
        nodes.append((job,1))
    
    return nodes

In [317]:
def make_problem(n_nodes_per_cluster, n_chargers,n_trips,print_=True):
    
    #random.seed(5918)
    
    problem = {}
    
    locations, cs, cs_locations = make_map_with_clusters(n_nodes_per_cluster=n_nodes_per_cluster,n_clusters=n_chargers)
    problem["Locations"] = locations
    charging_stations = list(zip(cs,cs_locations))
    
    problem["ChargingStations"] = charging_stations
    
    
    jobs = generate_jobs(locations,len(charging_stations))
    problem["Jobs"] = jobs
    
    
    nodes = all_nodes(jobs)
    
    problem["Nodes"] = nodes
    
    vessels = vessel_types()
    problem["Vessels"] = vessels
    
    problem["N_trips"] = n_trips
    
    
    costs = {"Fixed Penalty": 2000, #Penalty for each crew member outsourced. This makes a bigger job be more prenalized than a smaller job
             "PenaltyPerCrew": 750
            }
    problem["Costs"] = costs
    
    pp = preprocessing(problem)
    
    problem["PreProseccing"] = pp
    
    if print_:
        print("Jobs Information:")
        print("{:<15} {:<15}".format("Turbine ID", "Crew Needed"))
        for turbine_id, (crew_needed) in jobs.items():
            print("{:<15} {:<15}".format(turbine_id, crew_needed))
        
        print("Vessel Types Information:")
        print("{:<10} {:<15} {:<20} {:<25} {:<20} {:<20}".format("Vessel ID", "Battery Range", "Operational Cost", "Speed (Knots)", "Passenger Capacity", "Fixed Cost"))
        for specs in vessels.values():
            vessel_id, battery_range, operational_cost, speed, passenger_capacity,fixed_cost = specs
            print("{:<10} {:<15} {:<20} {:<25} {:<20} {:<20}".format(vessel_id, battery_range, operational_cost, speed, passenger_capacity,fixed_cost))
    
        
    
    
    return problem
    

In [318]:
def write_problem_to_file(problem, file_path):
    with open(file_path, 'w') as file:
        # Header and data for locations
        file.write("% number of locations\n")
        file.write(f"{len(problem['Locations'])}\n")
        file.write("% for each location: node, x, y\n")
        for node_id, (x, y) in problem['Locations'].items():
            file.write(f"{node_id}, {x}, {y}\n")
        # Number of vessels
        file.write("% number of vessels\n")
        file.write(f"{problem['N_trips']}\n")
        # Number and details of vessels
        file.write("% number of vessel types\n")
        file.write(f"{len(problem['Vessels'].keys())}\n")
        file.write("% for each vesseltype: vessel_id, battery range, operational cost, speed, passenger capacity, fixed cost\n")
        for vessel_id, specs in problem['Vessels'].items():
            file.write(f"{specs[0]}, {specs[1]}, {specs[2]}, {specs[3]}, {specs[4]}, {specs[5]} \n")
            
        # Header and data for charging stations
        file.write("% number of charging stations\n")
        file.write(f"{len(problem['ChargingStations'])}\n")
        file.write("% for each charging station: node, x, y\n")
        for station_id, (x, y) in problem['ChargingStations']:
            file.write(f"{station_id}, {x}, {y}\n")
            
        # Header and data for jobs
        file.write("% number of jobs\n")
        file.write(f"{len(problem['Jobs'])}\n")
        file.write("% for each job: node, crew needed\n")
        for node_id, crew_needed in problem['Jobs'].items():
            file.write(f"{node_id}, {crew_needed}\n")
        
        # Header and data for nodes
        file.write("% number of nodes\n")
        file.write(f"{len(problem['Nodes'])}\n")
        file.write("% for each node: node, binary\n")
        for node_id, binary in problem['Nodes']:
            file.write(f"{node_id}, {binary}\n")
            
        #Header and data for costs
        file.write("% costs\n")
        file.write("% fixed penalty\n")
        file.write(f"{problem['Costs']['Fixed Penalty']}\n")
        file.write("% penalty per crew\n")
        file.write(f"{problem['Costs']['PenaltyPerCrew']}\n")
        
        #Header and data for preprocessing
        file.write("% preprocessing\n")
         # Assuming 'preprocessing' is the correct dictionary key in your problem
        preprocessing = problem.get('PreProseccing', {})  # Using 'get' to avoid KeyError if missing
        for key, arr in preprocessing.items():
            file.write(f"% {key} \n")  # Writing the key
            for row in arr:
                # Writing each row of the array, values separated by commas
                row_str = ", ".join(map(str, row))
                file.write(f"{row_str}\n")


In [319]:


def parse_preprocessing_matrices(lines, n_nodes, is_tuple_array=False):
    """
    Parses lines into a numpy matrix, with special handling for arrays containing tuples.
    
    Args:
    - lines (list of str): The lines from the file to be parsed.
    - n_nodes (int): The number of nodes, which determines the matrix size (n_nodes x n_nodes).
    - is_tuple_array (bool): Flag to indicate if the array contains tuples, requiring special parsing.
    
    Returns:
    - np.array: The parsed matrix.
    """
    type = object if is_tuple_array else float
    matrix_data = np.empty((n_nodes, n_nodes),dtype=type)
    for i in range(n_nodes):
        line = lines[i]
        if is_tuple_array:
            row = make_tuple(line)
        else:
            row = [float(value) for value in line.split(', ')]
        for j in range(n_nodes):
            element = row[j]
            matrix_data[i, j] = element    
        
    return matrix_data, lines[n_nodes:]

def read_problem_from_file(file_path):
    problem = {
        "Locations": {},
        "Vessels": {},
        "ChargingStations": [],
        "Jobs": {},
        "Nodes": [],
        "Costs": {},
        "PreProseccing": {
            "PreferredCS": None,
            "Distances": None,
            "DistancesWithCharging": None
        }
    }

    with open(file_path, 'r') as file:
        lines = [line.strip() for line in file if line.strip() and not line.startswith('%')]
        
    lines_iter = iter(lines)
    n_locations = int(next(lines_iter))
    for _ in range(n_locations):
        node_id, x, y = map(int, next(lines_iter).split(', '))
        problem["Locations"][int(node_id)] = (int(x), int(y))
        
    problem["N_trips"] = int(next(lines_iter))

    n_vessels = int(next(lines_iter))
    for _ in range(n_vessels):
        parts = list(map(float, next(lines_iter).split(', ')))
        parts[0] = int(parts[0])
        problem["Vessels"][int(parts[0])] = parts

    n_charging_stations = int(next(lines_iter))
    for _ in range(n_charging_stations):
        station_id, x, y = map(int, next(lines_iter).split(', '))
        problem["ChargingStations"].append((int(station_id), (int(x), int(y))))

    n_jobs = int(next(lines_iter))
    for _ in range(n_jobs):
        job_id, crew_needed = map(int, next(lines_iter).split(', '))
        problem["Jobs"][int(job_id)] = int(crew_needed)

    n_nodes = int(next(lines_iter))
    for _ in range(n_nodes):  # Assuming twice the number of nodes for binary values
        parts = next(lines_iter).split(', ')
        node_id, binary = int(parts[0]), int(parts[1])
        problem["Nodes"].append((int(node_id), int(binary)))

    problem["Costs"]["Fixed Penalty"] = int(next(lines_iter))
    problem["Costs"]["PenaltyPerCrew"] = int(next(lines_iter))
    
    
    # Moving directly to preprocessing parsing for demonstration
    preprocessing_keys = ["PreferredCS", "Distances", "DistancesWithCharging"]
    remaining_lines = list(lines_iter)  # Get remaining lines for preprocessing
    for key in preprocessing_keys:
        if key == "PreferredCS":  # This key requires special handling for tuples
            matrix, remaining_lines = parse_preprocessing_matrices(remaining_lines, n_locations, is_tuple_array=True)
        else:
            matrix, remaining_lines = parse_preprocessing_matrices(remaining_lines, n_locations)
        problem["PreProseccing"][key] = matrix

    return problem


In [403]:
def load_problems(folder_path):
    problem_list = []
    files = os.listdir(folder_path)
    files = sorted(files, key=lambda name: int(re.match(r'(\d+)job', name).group(1)))
    for file in files:
        full_file_path = os.path.join(folder_path, file)
        problem = read_problem_from_file(full_file_path)
        problem_list.append(problem)
    return problem_list